# B-tree Implementation for Key-value Store
#### An example project illustring B-tree implementation in a key-value store, with numeric or non-numeric unique keys, range look-up, and functional and performance tests

In [77]:
# To structure code automatically
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

### Node Implementation

In [78]:
import bisect


class Node:
    def __init__(self, keys=None, values=None, children=None, parent=None):
        self.keys = keys or []
        self.values = values or []
        self.parent = parent
        self.set_children(children)

    def __len__(self):
        return len(self.values)

    def set_children(self, children):
        self.children = children or []
        for child in self.children:
            child.parent = self

    def is_leaf(self):
        return len(self.children) == 0

    def contains_key(self, key):
        return key in self.keys

    def get_value(self, key):
        for i, k in enumerate(self.keys):
            if k == key:
                return self.values[i]
        print(f'Key Error: "{key}" not found')
        return None

    def get_insert_index(self, key):
        return bisect.bisect(self.keys, key)

    def _update_entry(self, key, value):
        for i, k in enumerate(self.keys):
            if k == key:
                self.values[i] = value

    def insert_entry(self, key, value):
        if self.contains_key(key):
            self._update_entry(key, value)
        else:
            insert_index = self.get_insert_index(key)
            self.keys.insert(insert_index, key)
            self.values.insert(insert_index, value)
            return insert_index  # insert_index used for insert_child call on split_with_parent

    def split(self):
        if self.parent is None:
            return self._split_no_parent()
        return self._split_with_parent()

    def _split_no_parent(self):
        split_index = len(self) // 2
        key_to_move_up = self.keys[split_index]
        value_to_move_up = self.values[split_index]
        right_node = Node(
            self.keys[split_index + 1 :],
            self.values[split_index + 1 :],
            self.children[split_index + 1 :],
        )
        self.keys = self.keys[:split_index]
        self.values = self.values[:split_index]
        self.children = self.children[: split_index + 1]
        parent = Node([key_to_move_up], [value_to_move_up], [self, right_node])
        return parent

    def _insert_child(self, insert_index, child):
        self.children.insert(insert_index, child)
        child.parent = self

    def _split_with_parent(self):
        split_index = len(self) // 2
        key_to_move_up = self.keys[split_index]
        value_to_move_up = self.values[split_index]
        right_node = Node(
            self.keys[split_index + 1 :],
            self.values[split_index + 1 :],
            self.children[split_index + 1 :],
        )
        self.keys = self.keys[:split_index]
        self.values = self.values[:split_index]
        self.children = self.children[: split_index + 1]
        insert_index = self.parent.insert_entry(key_to_move_up, value_to_move_up)
        self.parent._insert_child(insert_index + 1, right_node)
        return self.parent

<IPython.core.display.Javascript object>

### B-tree Implementation

In [79]:
class BTree:
    def __init__(self, split_threshold=2):
        self.root = Node()
        self.height = 0
        self.size = 0
        self.split_threshold = split_threshold

    def __len__(self):
        return self.size

    def _add(self, current_node, key, value):
        if current_node.is_leaf():
            len_before = len(current_node)
            current_node.insert_entry(key, value)
            if len(current_node) > len_before:
                self.size += 1
        else:
            child_index = current_node.get_insert_index(key)
            self._add(current_node.children[child_index], key, value)
        if len(current_node) > self.split_threshold:
            parent = current_node.split()
            if current_node == self.root:
                self.root = parent
                self.height += 1

    def add(self, key, value):
        self._add(self.root, key, value)


    def _find_node(self, current_node, key):
        if current_node.contains_key(key):
            return current_node
        if current_node.is_leaf():
            return None
        child_index = current_node.get_insert_index(key)
        return self._find_node(current_node.children[child_index], key)

    def contains(self, key):
        node = self._find_node(self.root, key)
        return node is not None

    def get_value(self, key):
        node = self._find_node(self.root, key)
        if node is not None:
            for i, k in enumerate(node.keys):
                if k == key:
                    return node.values[i]
                
    def _range_query(
        self, current_node, range_start, range_end, min_key, max_key,
    ):
        if range_end < min_key or range_start > max_key:
            return []
        results = []
        for i, key in enumerate(current_node.keys):
            if range_start <= key <= range_end:
                results.append(current_node.values[i])
        if not current_node.is_leaf():
            for i, child in enumerate(current_node.children):
                new_min_key = current_node.keys[i - 1] if i > 0 else min_key
                new_max_key = current_node.keys[i] if i < len(current_node) else max_key
                results += self._range_query(
                    child, range_start, range_end, new_min_key, new_max_key
                )
        return results

    def range_query(self, range_start, range_end):
        return self._range_query(
            self.root, range_start, range_end, float("-inf"), float("inf")
        )


<IPython.core.display.Javascript object>

In [80]:
bt = BTree()
for i in range(10):
    bt.add(i, i)

<IPython.core.display.Javascript object>

In [81]:
bt.range_query(2, 4)

[3, 2, 4]

<IPython.core.display.Javascript object>

In [82]:
bt.range_query(2, 2)

[2]

<IPython.core.display.Javascript object>

In [83]:
bt.contains(11)

False

<IPython.core.display.Javascript object>

In [84]:
bt.get_value(9)

9

<IPython.core.display.Javascript object>

In [85]:
bt.get_value(9)

9

<IPython.core.display.Javascript object>